In [ ]:
!pip install -q transformers datasets scikit-learn

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

CSV_PATH = "/content/hate_speech_hindi_final.csv"

df = pd.read_csv(CSV_PATH)
print(df.head())
print(df.columns)

                                                text  \
0  मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...   
1  सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...   
2  सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...   
3  @prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...   
4  #unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...   

                                          clean_text  \
0  मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के...   
1  सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...   
2  सुशांत ने जो बिजनेस डील जून को की थी वो डील दी...   
3  साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते ...   
4  unlockguidelines अनलॉक के लिए गाइडलाइन्स जारी ...   

                                   text_no_stopwords  defamation  hate  \
0  मेरे देश हिन्दु निराले है पक्के राम भक्त है बा...           0     1   
1  सरकार हमेशा किसानों कमाई बढ़ाने नईनई स्कीमें ल...           0     0   
2  सुशांत बिजनेस डील जून थी वो डील दीपेश सुशांत ह...           0     0   
3  साले जेएनयू छाप कमिने लोग ह

In [ ]:
# Multi-label columns
label_cols = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
num_labels = len(label_cols)

# Ensure labels are int (0/1)
df[label_cols] = df[label_cols].astype(int)

TEXT_COLUMN = "text_no_stopwords"

# Train / validation split
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df[label_cols].values.argmax(axis=1)
)

print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)

Train shape: (31892, 8)
Val shape: (7973, 8)


In [ ]:
model_name = "jplu/tf-xlm-roberta-base"  # TF checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_length = 128

# Convert texts to lists of strings
train_texts = train_df[TEXT_COLUMN].astype(str).tolist()
val_texts   = val_df[TEXT_COLUMN].astype(str).tolist()

# Tokenize
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding="max_length",
    max_length=max_length
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding="max_length",
    max_length=max_length
)

# Labels as float32 arrays
train_labels = train_df[label_cols].values.astype("float32")
val_labels   = val_df[label_cols].values.astype("float32")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/512 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

In [ ]:
def create_tf_dataset(encodings, labels, batch_size=16, shuffle=False):
    inputs = {
        "input_ids":      np.array(encodings["input_ids"]),
        "attention_mask": np.array(encodings["attention_mask"])
    }
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
    if shuffle:
        dataset = dataset.shuffle(len(labels))
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 16

train_dataset = create_tf_dataset(train_encodings, train_labels, batch_size=batch_size, shuffle=True)
val_dataset   = create_tf_dataset(val_encodings,   val_labels,   batch_size=batch_size, shuffle=False)

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels
)

config.problem_type = "multi_label_classification"

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config
)

tf_model.h5:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

Some layers of TFXLMRobertaForSequenceClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

metrics = [
    tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", threshold=0.5)
]

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

model.summary()

Model: "tfxlm_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFXLMRobertaMainL  multiple                  277453056 
 ayer)                                                           
                                                                 
 classifier (TFXLMRobertaCl  multiple                  594437    
 assificationHead)                                               
                                                                 
Total params: 278047493 (1.04 GB)
Trainable params: 278047493 (1.04 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
epochs = 5

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

Epoch 1/5
1994/1994 [==============================] - 1016s 476ms/step - loss: 0.5025 - binary_accuracy: 0.7615 - val_loss: 0.4548 - val_binary_accuracy: 0.7815
Epoch 2/5
1994/1994 [==============================] - 918s 461ms/step - loss: 0.4413 - binary_accuracy: 0.7872 - val_loss: 0.3900 - val_binary_accuracy: 0.8117
Epoch 3/5
1994/1994 [==============================] - 918s 460ms/step - loss: 0.3655 - binary_accuracy: 0.8291 - val_loss: 0.3420 - val_binary_accuracy: 0.8500
Epoch 4/5
1994/1994 [==============================] - 917s 460ms/step - loss: 0.3064 - binary_accuracy: 0.8616 - val_loss: 0.2931 - val_binary_accuracy: 0.8720
Epoch 5/5
1994/1994 [==============================] - 917s 460ms/step - loss: 0.2713 - binary_accuracy: 0.8790 - val_loss: 0.2743 - val_binary_accuracy: 0.8813


In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
import tensorflow as tf
import numpy as np

# Predict on validation dataset
pred_outputs = model.predict(val_dataset)

# Extract logits correctly
logits = pred_outputs.logits if hasattr(pred_outputs, "logits") else pred_outputs

# Convert logits -> probabilities
probs = tf.sigmoid(logits).numpy()

# Multi-label threshold
threshold = 0.5
y_pred = (probs >= threshold).astype(int)

# Ground truth
y_true = val_labels

print("\n🔹 Shapes check:")
print("Logits:", logits.shape)
print("Pred:", y_pred.shape)
print("True:", y_true.shape)

# ---- ACCURACY ----
subset_accuracy = accuracy_score(y_true, y_pred)
print(f"\n Exact Match Accuracy: {subset_accuracy:.4f}")

# ---- F1 Scores ----
macro_f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
micro_f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

print(f" Macro F1 Score: {macro_f1:.4f}")
print(f" Micro F1 Score: {micro_f1:.4f}")

# ---- Full classification report ----
print("\n Classification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=label_cols,
    zero_division=0
))


499/499 [==============================] - 72s 139ms/step

🔹 Shapes check:
Logits: (7973, 5)
Pred: (7973, 5)
True: (7973, 5)

 Exact Match Accuracy: 0.7144
 Macro F1 Score: 0.7481
 Micro F1 Score: 0.7481

 Classification Report:

              precision    recall  f1-score   support

  defamation       0.94      0.62      0.74      2000
        hate       0.76      0.63      0.69      1996
    non-hate       0.75      0.71      0.73      2000
    violence       0.81      0.74      0.77      2004
      vulgar       0.89      0.74      0.80      1975

   micro avg       0.82      0.69      0.75      9975
   macro avg       0.83      0.69      0.75      9975
weighted avg       0.83      0.69      0.75      9975
 samples avg       0.80      0.77      0.78      9975

